In [30]:
import arrow
import datetime
import numpy as np
import os
from pathlib import Path
import shlex
import subprocess
import xarray as xr

# Choose Date Range

In [47]:
startdate = datetime.datetime(2014, 11, 1, 1)
enddate = datetime.datetime(2014, 11, 13, 23)

# Set up Directory Etc

In [3]:
thedir = '/data/dlatorne/tmp-rpn-to-gem-lam/'
filehead = 'gemlam_'
existence = {}

# Interpolate Any Missing Files

## Check for Missing Files

In [48]:
for hour in arrow.Arrow.range('hour', startdate, enddate):
    file_suffix =(
        f'y{hour.year}m{hour.month:02d}d{hour.day:02d}'
        f'_{hour.hour:03d}')
    fname = Path(
        f"{filehead}"
        f'{file_suffix}.nc'
    )
    filename = thedir / fname
    existence[hour] = os.path.isfile(filename)
    if not existence[hour]:
        print (hour)

There are none in this November set, so onwards and we'll come back to this.

# Calculate Solar (average) and Precip (difference)

In [49]:
for hour in arrow.Arrow.range('hour', startdate, enddate):
    prevhour = hour.shift(hours=-1)
    file_suffix = (
        f'y{hour.year}m{hour.month:02d}d{hour.day:02d}'
        f'_{hour.hour:03d}')
    prevfile_suffix = (
        f'y{prevhour.year}m{prevhour.month:02d}d{prevhour.day:02d}'
        f'_{prevhour.hour:03d}')
    fname = Path(
        f"{filehead}"
        f'{file_suffix}.nc'
    )
    prevfname = Path(
        f"{filehead}"
        f'{prevfile_suffix}.nc'
    )
    # make a copy of the file in the local directory
    filename = thedir / fname
    cmd = (
        f'cp '
        f'{filename}'
        f' .'
        )
    subprocess.call(shlex.split(cmd))
                
    # calculate the solar    
    prevfilename = thedir / prevfname
    outfile = 'solar.nc'
    
    cmd = (
        f'ncra -v solar '
        f'{filename} ' 
        f'{prevfilename} '
        f'-O -o '
        f'{outfile}')
    subprocess.call(shlex.split(cmd))

    # insert new solar in our file
    cmd = (
        f'ncks -A -v solar '
        f'solar.nc ' 
        f'{fname} '
       )
    subprocess.call(shlex.split(cmd))

    
    # calculate the precip
    
    # check if we are accumulating or at the first file
    prevsum = 'prevsumprecip.nc'
    currsum = 'sumprecip.nc'
    prevsumtxt = 'prevsumprecip.txt'
    cmd = (
        f'ncwa -v precip '
        f'{prevfilename} ' 
        f'-O -o'
        f'{prevsum}')
    subprocess.call(shlex.split(cmd))
#    cmd = (
#        f'ncdump -v precip '
#        f'{prevsum} ' 
#        f' | grep "precip ="'
#        )
#    print(subprocess.check_output(shlex.split(cmd, posix=False)))
    cmd = (
        f'ncwa -v precip '
        f'{filename} ' 
        f'-O -o'
        f'{currsum}')
    subprocess.call(shlex.split(cmd))
    print (xr.open_dataset(currsum)['precip'] ,
        xr.open_dataset(prevsum)['precip'])
    if (xr.open_dataset(currsum)['precip'] >
        xr.open_dataset(prevsum)['precip']) :
    
        outfile = 'precip.nc'
        print (filename, 'doing it')
    
        cmd = (
            f'ncdiff -v precip '
            f'{filename} ' 
            f'{prevfilename} '
            f'-O -o'
            f'{outfile}')
        subprocess.call(shlex.split(cmd))
    
        # insert new precip in file
        cmd = (
        f'ncks -A -v precip '
        f'precip.nc ' 
        f'{fname} '
       )
        subprocess.call(shlex.split(cmd))

<xarray.DataArray 'precip' ()>
array(0.006747, dtype=float32)
Attributes:
    level:          surface
    long_name:      Total Precipitation
    standard_name:  precipitation_flux
    units:          kg/m^2/hr
    cell_methods:   time_counter, y, x: mean <xarray.DataArray 'precip' ()>
array(0.005711, dtype=float32)
Attributes:
    level:          surface
    long_name:      Total Precipitation
    standard_name:  precipitation_flux
    units:          kg/m^2/hr
    cell_methods:   time_counter, y, x: mean
/data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2014m11d01_001.nc doing it
<xarray.DataArray 'precip' ()>
array(0.006747, dtype=float32)
Attributes:
    level:          surface
    long_name:      Total Precipitation
    standard_name:  precipitation_flux
    units:          kg/m^2/hr
    cell_methods:   time_counter, y, x: mean <xarray.DataArray 'precip' ()>
array(0.005711, dtype=float32)
Attributes:
    level:          surface
    long_name:      Total Precipitation
    standard_name:  p

In [ ]:
)filename